In [1]:
import sqlite3
import io
import csv
import os
from dotenv import load_dotenv
from openai import OpenAI
import json
os.chdir("..")
from chatgpt_api import chat_prompt
from chatgpt_api.api import schema_info, db_schema




load_dotenv()
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
def load_schema_from_json(file_path: str) -> dict:
    try:
        with open(file_path, 'r') as f:
            schema = json.load(f)
        return schema
    except FileNotFoundError:
        raise FileNotFoundError
    except json.JSONDecodeError as e:
        e
db_schema = load_schema_from_json('chatgpt_api/schema.json')
def format_schema_for_gpt(schema: dict) -> str:
    try:
        formatted_schema = ""
        for table in schema.get('tables', []):
            table_name = table['url'].replace('.tsv', '')
            columns = table['tableSchema']['columns']
            column_list = ", ".join([
                f"{col['name']} {col['datatype']['base'].upper()}"
                for col in columns
            ])
            formatted_schema += f"{table_name}: {column_list}\n"
        return formatted_schema
    except Exception as e:
        raise e
schema_info = format_schema_for_gpt(db_schema) 

def execute_sql(query: str) -> str:
    conn = None
    try:
        conn = sqlite3.connect('sqlite/nport.db')
        cursor = conn.cursor()

        # Execute the query with a timeout
        cursor.execute(query)

        # Fetch column names and rows
        columns = [description[0] for description in cursor.description]
        rows = cursor.fetchall()

        # Convert the results to CSV
        output = io.StringIO()
        writer = csv.writer(output)
        writer.writerow(columns)
        writer.writerows(rows)
        csv_data = output.getvalue()
        output.close()

        return csv_data
    except sqlite3.Error as e:
        print(f"Database error: {str(e)}")
        raise e
    except Exception as e:
        print(f"Error executing SQL: {str(e)}")
        raise e
    finally:
        if conn:
            conn.close()

def get_prompt() -> str:
    try:
        return chat_prompt.full_prompt
    except (ImportError, AttributeError) as e:
        print(f"Failed to load chat prompt: {str(e)}")
        return ""
def generate_sql(question: str) -> str:
    
    if not db_schema:
        raise Exception(
            status_code=500,
            detail="Database schema not loaded"
        )

    #schema_info = format_schema_for_gpt(db_schema) 
    prompt = f"""
    ```
    OVERALL TASK:
    I will provide a database schema, generate an SQL query that retrieves from the database the answer to this question: {question}
    ```
    """ + get_prompt()
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "system",
                    "content": "You are a financial database assistant that generates SQL queries based on natural language questions about financial data."
                },
                {"role": "user", "content": prompt}
            ],
            tools = [
                {
                    "type": "function",
                    "function": {
                        "name": "sql_query",
                        "description": "Execute SQL queries against the database schema which includes tables like FUND_REPORTED_INFO (containing SERIES_NAME, TOTAL_ASSETS) and REGISTRANT (containing REGISTRANT_NAME). Use these tables to analyze fund data.",
                        "parameters": {
                            "type": "object",
                            "properties": {
                                "query": {
                                    "type": "string",
                                    "description": "The SQL query to execute. Example: To find top PIMCO funds, use: SELECT SERIES_NAME, TOTAL_ASSETS FROM FUND_REPORTED_INFO JOIN REGISTRANT ON FUND_REPORTED_INFO.ACCESSION_NUMBER = REGISTRANT.ACCESSION_NUMBER WHERE REGISTRANT_NAME LIKE '%PIMCO%' ORDER BY TOTAL_ASSETS DESC LIMIT 5"
                                }
                            },
                            "required": ["query"]
                        }
                    },
                    "schema": schema_info # this is where the schema is entered
                }
            ], 
        tool_choice={"type": "function", "function": {"name": "sql_query"}}  # force the model to use the SQL function
        )

        function_call = response.choices[0].message.tool_calls[0].function
        if function_call.name == "sql_query":
            # openai returns function arguments as a JSON string, so we need to parse it
            # The query will be in the "query" field of the parsed JSON
            sql_query = json.loads(function_call.arguments)["query"]
            return sql_query
        else:
            raise ValueError(f"Unexpected function call: {function_call.name}")

    except Exception as e:
        raise e(
            status_code=500,
            detail=f"Error generating SQL: {str(e)}"
    )

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='C:\\Users\\User\\anaconda3\\Library\\ssl\\cacert.pem'
INFO:chatgpt_api.api:OpenAI API key loaded successfully
DEBUG:chatgpt_api.api:Attempting to load schema from chatgpt_api/schema.json
INFO:chatgpt_api.api:Schema loaded successfully


Expected schema path: chatgpt_api/schema.json


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='C:\\Users\\User\\anaconda3\\Library\\ssl\\cacert.pem'


In [2]:
def compare_csv(ground_truth_query: str, llm_query: str):
    ## let LLM stack query the database

    
    generated_sql = generate_sql(llm_query)

    llm_csv = execute_sql(generated_sql)  
    ground_truth_csv = execute_sql(ground_truth_query)

    ## compare results    
    try:
        diff=compare_csv_strings(ground_truth_csv,llm_csv)
            # compare df
        #df_ground_truth = pd.read_csv(io.StringIO(ground_truth_csv))
        #df_llm = pd.read_csv(io.StringIO(llm_csv))
        #diff = df_ground_truth.equals(df_llm)

        if diff:
            #return {"result": "CSV outputs match perfectly."}
            print("CSV outputs match perfectly.")
            return True
        else:
            #total_cells = df_llm.size
            #diff_cells = diff.count().sum()
            #difference_percentage = (diff_cells / total_cells) * 100
            #print(f"Mismatch found. Difference percentage: {difference_percentage:.2f}%")
            print("Mismatch found.")
            return False
    except Exception as e:
        raise e(
            status_code=500,
            detail=f"Error comparing CSVs: {str(e)}"
        )
    
def compare_csv_strings(csv_data1: str, csv_data2: str) -> bool:
    # Use io.StringIO to read the CSV strings as file-like objects
    csv_file1 = io.StringIO(csv_data1)
    csv_file2 = io.StringIO(csv_data2)
    
    # Create CSV readers for each CSV string
    reader1 = csv.reader(csv_file1)
    reader2 = csv.reader(csv_file2)
    
    # Compare rows one by one
    for row1, row2 in zip(reader1, reader2):
        if row1 != row2:
            return False  # Rows are different
    
    # Check if there are extra rows in either file
    try:
        next(reader1)
        return False  # Extra rows in csv_data1
    except StopIteration:
        pass

    try:
        next(reader2)
        return False  # Extra rows in csv_data2
    except StopIteration:
        pass

    return True  # CSVs are identical



In [3]:
import csv
import json

# Load data from query_summary.csv
def load_queries(input_file):
    llm_query = []
    ground_truth_query = []
    with open(input_file, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            llm_query.append(row["Question"])
            ground_truth_query.append(row["SQL"])
    return llm_query, ground_truth_query

# Save arrays to file
def save_queries_to_file(file_path, llm_query, ground_truth_query):
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump({"llm_query": llm_query, "ground_truth_query": ground_truth_query}, file)

# Load arrays from file
def load_queries_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
        return data["llm_query"], data["ground_truth_query"]

# Write comparison results to output file
def write_to_output(file_path, ground_truth, llm, result):
    with open(file_path, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([ground_truth, llm, result])

# Main script logic
def process_queries(input_file, temp_file, output_file):
    # If temp file doesn't exist, load queries from CSV
    try:
        llm_query, ground_truth_query = load_queries_from_file(temp_file)
    except FileNotFoundError:
        llm_query, ground_truth_query = load_queries(input_file)
        save_queries_to_file(temp_file, llm_query, ground_truth_query)

    # Prepare output file with a header if starting fresh
    if not os.path.exists(output_file):
        with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(["Ground_Truth_Query", "LLM_Query", "Comparison_Result"])

    # Process queries
    while llm_query and ground_truth_query:
        gt_query = ground_truth_query.pop(0)
        llm_query_item = llm_query.pop(0)

        # Compare queries and write results
        try:
            result = compare_csv(gt_query, llm_query_item)
            write_to_output(output_file, gt_query, llm_query_item, result)
        except Exception as e:
            print(f"Error comparing queries: {e}")
            write_to_output(output_file, gt_query, llm_query_item, "Error")

        # Save the remaining queries back to the temp file
        save_queries_to_file(temp_file, llm_query, ground_truth_query)


In [4]:
print(os.getcwd())

d:\GithubRepos\PIMCO-Text2SQL


In [ ]:
input_file = "query_summary.csv"
temp_file = "temp_queries.json"  # Temporary file to store remaining queries
output_file = "original_accuracy_120.csv"

process_queries(input_file, temp_file, output_file)


DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'You are a financial database assistant that generates SQL queries based on natural language questions about financial data.'}, {'role': 'user', 'content': '\n    ```\n    OVERALL TASK:\n    I will provide a database schema, generate an SQL query that retrieves from the database the answer to this question: Show me the top 5 largest funds by total assets\n    ```\n    \n```\nTask Description:\nThe task is to transform the natural language query into a SQL query for SQLite database.\nThis involves parsing the intent of the query and understanding the structure of the data to generate an appropriate SQL command.\n```\n\n```\nDatabase Overview:\n- The Database combines information from 30 tables of the NPORT dataset from quarter 4 of 2019 to quarter 3 of 2024.\n- The data includes a comprehensive view of fund-level information, h

CSV outputs match perfectly.


DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Tue, 26 Nov 2024 21:31:14 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'user-annfuni26pdtuawdwdj6zorw'), (b'openai-processing-ms', b'751'), (b'openai-version', b'2020-10-01'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'2000000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'1994019'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset-tokens', b'179ms'), (b'x-request-id', b'req_dbc22238ae3ca48abe9787df5ffda446'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'CF-Cache-Status', b'DYNAMIC'), (b'X-Content-Type-Options', b'nosniff'), (b'Server', b'cloudflare'), (b'CF-RAY', b'8e8cfbf4eeee7c80-LAX'), (b'Content-Encoding', b'

Mismatch found.


DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Tue, 26 Nov 2024 21:31:15 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'user-annfuni26pdtuawdwdj6zorw'), (b'openai-processing-ms', b'718'), (b'openai-version', b'2020-10-01'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'2000000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'1994019'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset-tokens', b'179ms'), (b'x-request-id', b'req_14506d1b8ef6cad99c723fdff208d3f8'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'CF-Cache-Status', b'DYNAMIC'), (b'X-Content-Type-Options', b'nosniff'), (b'Server', b'cloudflare'), (b'CF-RAY', b'8e8cfbfb0f0a7c80-LAX'), (b'Content-Encoding', b'

...

Mismatch found.


DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Tue, 26 Nov 2024 21:36:08 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'user-annfuni26pdtuawdwdj6zorw'), (b'openai-processing-ms', b'3264'), (b'openai-version', b'2020-10-01'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'2000000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'1994019'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset-tokens', b'179ms'), (b'x-request-id', b'req_db280150b5ac6da8b41176241311625e'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'CF-Cache-Status', b'DYNAMIC'), (b'X-Content-Type-Options', b'nosniff'), (b'Server', b'cloudflare'), (b'CF-RAY', b'8e8d03111b0b7c80-LAX'), (b'Content-Encoding', b

Mismatch found.


DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Tue, 26 Nov 2024 21:36:10 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'user-annfuni26pdtuawdwdj6zorw'), (b'openai-processing-ms', b'1873'), (b'openai-version', b'2020-10-01'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'2000000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'1994021'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset-tokens', b'179ms'), (b'x-request-id', b'req_7f8b66a62f4d46932b306afebad547f0'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'CF-Cache-Status', b'DYNAMIC'), (b'X-Content-Type-Options', b'nosniff'), (b'Server', b'cloudflare'), (b'CF-RAY', b'8e8d0326aa3a7c80-LAX'), (b'Content-Encoding', b

Database error: no such column: DEBT_SECURITY.TOTAL_LEVERAGE
Error comparing queries: no such column: DEBT_SECURITY.TOTAL_LEVERAGE
